<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/DenseNet121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 

     |████████████████████████████████| 1.9 MB 37.9 MB/s 
     |████████████████████████████████| 182 kB 69.6 MB/s 
     |████████████████████████████████| 166 kB 64.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 72.4 MB/s 
     |████████████████████████████████| 162 kB 78.8 MB/s 
     |████████████████████████████████| 162 kB 73.9 MB/s 
     |████████████████████████████████| 158 kB 70.4 MB/s 
     |████████████████████████████████| 157 kB 68.9 MB/s 
     |████████████████████████████████| 157 kB 75.1 MB/s 
     |████████████████████████████████| 157 kB 78.2 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 157 kB 79.0 MB/s 
     |████████████████████████████████| 157 kB 84.4 MB/s 
     |████████████████████████████████| 157 kB 77.6 MB/s 
     |████████████████████████████████| 156 kB 76.8 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [7]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [8]:
densenet121 = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)

model_densenet121=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  densenet121,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
densenet121.trainable=False
model_densenet121.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 1024)              7037504   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 5125      
                                                                 
Total params: 7,042,629
Trainable params: 5,125
Non-trainable params: 7,037,504
_________________________________________________________________


In [9]:
model_densenet121.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [10]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "densenet121",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [11]:
%%time
epochs=40
history = model_densenet121.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.2245 - accuracy: 0.5199

wandb: Adding directory to artifact (/content/wandb/run-20221107_091412-2ru5tnq2/files/model-best)... Done. 0.2s


22/22 [==============================] - 333s 11s/step - loss: 1.2245 - accuracy: 0.5199 - val_loss: 0.8569 - val_accuracy: 0.6743
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.6829 - accuracy: 0.7685

wandb: Adding directory to artifact (/content/wandb/run-20221107_091412-2ru5tnq2/files/model-best)... Done. 0.2s


...

22/22 [==============================] - 48s 2s/step - loss: 0.0479 - accuracy: 0.9943 - val_loss: 0.0624 - val_accuracy: 0.9886
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.0478 - accuracy: 0.9886

wandb: Adding directory to artifact (/content/wandb/run-20221107_091412-2ru5tnq2/files/model-best)... Done. 0.2s


22/22 [==============================] - 48s 2s/step - loss: 0.0478 - accuracy: 0.9886 - val_loss: 0.0601 - val_accuracy: 0.9943
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 0.0397 - accuracy: 0.9972

wandb: Adding directory to artifact (/content/wandb/run-20221107_091412-2ru5tnq2/files/model-best)... Done. 0.4s


22/22 [==============================] - 48s 2s/step - loss: 0.0397 - accuracy: 0.9972 - val_loss: 0.0581 - val_accuracy: 0.9886
CPU times: user 27min 45s, sys: 59.6 s, total: 28min 44s
Wall time: 44min 44s


In [12]:
wandb.finish()

accuracy,▁▅▆▇▇▇▇▇████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▇▇▇▇▇▇▇▇▇▇███████████████████████████
val_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.99716
best_epoch,39
best_val_loss,0.05812
epoch,39
loss,0.03973
val_accuracy,0.98857


In [13]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
